## 환경 설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Chroma 특정 Collection 제거

In [63]:
import os
print(os.path.abspath("../vs"))

c:\Users\CNXK\Documents\ktrip-bot\vs


In [65]:
from chromadb import Client
from chromadb.config import Settings

client = Client(Settings(persist_directory="../vs"))
collections = client.list_collections()
collections

[]

In [59]:
# 삭제할 컬렉션 이름
collection_name = ""

# 해당 컬렉션이 존재하면 삭제
try:
    client.delete_collection(name=collection_name)
    print(f"✅ 컬렉션 '{collection_name}' 삭제 완료.")
except Exception as e:
    print(f"⚠️ 컬렉션 삭제 실패: {e}")


⚠️ 컬렉션 삭제 실패: Collection [category] does not exists


### Callback 정의

In [2]:
from langfuse.langchain import CallbackHandler

langfuse_handler = CallbackHandler()

In [3]:
import time
import logging
from datetime import datetime
from typing import Dict, List, Any, Optional
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.outputs import LLMResult

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

class PerformanceMonitoringCallback(BaseCallbackHandler):
    """LLM 호출 성능을 모니터링하는 콜백 핸들러"""
    
    def __init__(self):
        self.start_time: Optional[float] = None
        self.token_usage: Dict[str, Any] = {}
        self.call_count: int = 0
        
    def on_llm_start(
        self, 
        serialized: Dict[str, Any], 
        prompts: List[str], 
        **kwargs: Any
    ) -> None:
        """LLM 호출이 시작될 때 호출"""
        self.start_time = time.time()
        self.call_count += 1
        print(f"🚀 LLM 호출 #{self.call_count} 시작 - {datetime.now().strftime('%H:%M:%S')}")
        
        # 첫 번째 프롬프트의 길이 확인
        if prompts:
            print(f"📝 프롬프트 길이: {len(prompts[0])} 문자")
        
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """LLM 호출이 완료될 때 호출"""
        if self.start_time:
            duration = time.time() - self.start_time
            print(f"✅ LLM 호출 완료 - 소요시간: {duration:.2f}초")
            
            # 토큰 사용량 추적
            if response.generations:
                generation = response.generations[0][0]
                
                # usage_metadata를 우선 확인 
                if hasattr(generation, 'usage_metadata') and generation.usage_metadata:
                    usage = generation.usage_metadata
                    print(f"🔢 토큰 사용량: {usage}")
                    self.token_usage = usage
                    
                # 구버전 호환성을 위한 llm_output 확인
                elif hasattr(response, 'llm_output') and response.llm_output:
                    usage = response.llm_output.get('token_usage', {})
                    if usage:
                        print(f"🔢 토큰 사용량: {usage}")
                        self.token_usage = usage
                        
                # 응답 길이 체크
                if hasattr(generation, 'text'):
                    response_text = generation.text
                    print(f"📊 응답 길이: {len(response_text)} 문자")
        
    def on_llm_error(self, error: Exception, **kwargs: Any) -> None:
        """LLM 호출에서 오류가 발생할 때 호출"""
        print(f"❌ LLM 호출 오류: {str(error)}")
        
    def get_statistics(self) -> Dict[str, Any]:
        """현재까지의 통계 정보를 반환"""
        return {
            "total_calls": self.call_count,
            "last_token_usage": self.token_usage
        }

---

# 사용자 입력 정의

## 1. 여행 간편 조건

In [70]:
from enum import Enum, StrEnum, auto

In [71]:
users = [{},{},{},{},{}] # 사용자 예시

### 누구와 여행하시나요?

In [72]:
class Companion(StrEnum):
    """Type of travel companion."""
    Solo = auto()            # Traveling alone
    Couple = auto()          # Traveling with a partner
    Family = auto()          # Family travel
    Friends = auto()         # Trip with friends

In [73]:
users[0]["companion"] = Companion.Family.value
users[1]["companion"] = Companion.Couple.value
users[2]["companion"] = Companion.Family.value
users[3]["companion"] = Companion.Solo.value
users[4]["companion"] = Companion.Friends.value

### 어떤 분위기의 여행을 원하세요?

In [74]:
class TravelPurpose(StrEnum):
    Relaxation = auto()         # 휴식 / 힐링
    Nature = auto()             # 자연 감상
    Adventure = auto()          # 레저 스포츠 / 액티비티
    Culture = auto()            # 역사 / 전통 문화 체험
    Festival = auto()           # 축제 / 공연 / 이벤트
    Gourmet = auto()            # 맛집 탐방 / 미식 여행
    Family = auto()             # 가족 중심 여행 (아이 포함)
    Romance = auto()            # 연인과의 로맨틱 여행
    CityTour = auto()           # 도시 탐방
    Shopping = auto()           # 쇼핑

In [75]:
users[0]["purpose"] = [TravelPurpose.Nature.value, 
                       TravelPurpose.Culture.value,
                       TravelPurpose.Family.value]
users[1]["purpose"] = [TravelPurpose.Relaxation.value,
                       TravelPurpose.Gourmet.value,
                       TravelPurpose.Romance.value]
users[2]["purpose"] = [TravelPurpose.Culture.value,
                       TravelPurpose.Family.value]
users[3]["purpose"] = [TravelPurpose.Relaxation.value]
users[4]["purpose"] = [TravelPurpose.Adventure.value,
                       TravelPurpose.Gourmet.value,
                       TravelPurpose.Shopping.value]

### 언제부터 언제까지 가시나요?

In [77]:
users[0]["start_time"] = "20250801 09:00"
users[0]["end_time"] = "20250804 16:00"

users[1]["start_time"] = "20250910 15:00"
users[1]["end_time"] = "20250911 20:00"

users[2]["start_time"] = "20251003 08:30"
users[2]["end_time"] = "20251005 18:00"

users[3]["start_time"] = "20250810 10:00"
users[3]["end_time"] = "20250817 16:00"

users[4]["start_time"] = "20250816 08:00"
users[4]["end_time"] = "20250818 18:00"

### 추가로 요청하실 내용이 있다면 자유롭게 적어주세요

In [78]:
users[0]["notes"] = "아이와 함께 체험할 수 있는 자연놀이터&아이랑 같이 쉬기 좋은 숙소 위주"
users[1]["notes"] = "별을 볼 수 있는 조용한 야외 공간\n혼잡하지 않은 한적한 곳 선호"
users[2]["notes"] = "아이와 어르신 모두 즐길 수 있는 전통문화 체험이 필요합니다. 편안한 숙소와 쉬는 시간 많은 일정이 좋습니다."
users[3]["notes"] = "조용한 바닷가 산책과 책 읽기 좋은 숙소, 휴식 위주의 일정, 혼자서 조용히 자연을 느낄 수 있는 장소 중심"
users[4]["notes"] = "익스트림 스포츠와 액티비티(짚라인, 패러글라이딩, 서핑)가 밀집된 지역 중심으로 추천"

## 2. 상세 조건

### 총 몇 명이 함께 여행하시나요? (아이, 어르신 포함)

In [85]:
users[0]["num_people"] = 3
users[2]["num_people"] = 7
users[4]["num_people"] = 3

### 아이(영유아 또는 어린이)가 포함되어 있나요?

In [86]:
users[0]["has_infant"] = True
users[2]["has_infant"] = True
users[4]["has_infant"] = False

### 어르신(고령자)이 포함되어 있나요?

In [87]:
users[0]["has_elder"] = False
users[2]["has_elder"] = True
users[4]["has_elder"] = False

### 반려동물(강아지/고양이 등)도 함께 여행하시나요?

In [88]:
users[0]["has_pet"] = True
users[2]["has_pet"] = False
users[4]["has_pet"] = True

### 여행 중엔 어떤 교통수단을 사용하실 계획인가요?

In [89]:
class Transport(StrEnum):
    """Preferred transport method."""
    Car = auto()             # Private vehicle
    PublicTransport = auto() # Bus, subway, train, etc.
    Walking = auto()         # On foot
    Bike = auto()            # Bicycle or similar

In [90]:
users[0]["transport"] = Transport.Car.value
users[2]["transport"] = Transport.Car.value
users[4]["transport"] = Transport.PublicTransport.value

### 1인당 여행 예산은 어느 정도로 생각하고 계신가요?

In [91]:
users[0]["per_person_budget"] = "200,000 KRW"
users[2]["per_person_budget"] = "150,000 KRW"
users[4]["per_person_budget"] = "100,000 KRW"

---

# 관광지 정보 탐색

## 1. 관광지 카테고리 조회

### 1\) 카테고리 벡터화

- 사용자 여행 조건을 의미 기반 및 키워드 중심으로 표현하여, 맞춤형 추천과 탐색이 가능하도록 벡터화하는 작업임

In [82]:
import ast
import re

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_community.utilities import SQLDatabase
from langchain_community.retrievers import BM25Retriever

from langchain.retrievers import EnsembleRetriever


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

db = SQLDatabase.from_uri(
    "sqlite:///../kto_data.db"
)

# 벡터화 작업 리스트
sql = """
SELECT DISTINCT lclsSystm3Nm
FROM lclsSystmCode2
WHERE lclsSystm1Cd <> 'AC' -- 숙박
AND lclsSystm2Cd <> 'VE11' -- 교통시설
-- AND lang = 'Eng'
"""
data = query_as_list(db, sql)

# 임베딩 모델 정의
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 벡터 저장소 및 검색기 생성
chroma_db = Chroma(
    collection_name="Subcategory",
    embedding_function=embeddings,
    persist_directory="../vs",
)

data_add = False # 데이터를 벡터스토어에 저장 여부 * 나중에 언어별 추가 필요
if data_add:
    # 데이터를 벡터스토어에 저장
    chroma_db.add_texts(data)

### 2\) 검색기 정의

In [83]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

retriever = chroma_db.as_retriever(
    # search_type="mmr",
    search_kwargs={"k": 5}
)
bm25_retriever = BM25Retriever.from_texts(
    texts=data,
    preprocess_func=lambda x: [t.form for t in kiwi.tokenize(x)],
    k=5
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever], 
    weights=[0.5, 0.5]          
)

In [93]:
import json
from pprint import pprint

# 테스트
for idx, user in enumerate(users):
    print("\n\n[여행 조건]")
    pprint(user)
    retrieved = ensemble_retriever.invoke(json.dumps(user))
    print("\n[카테고리 검색 결과]")
    users[idx]['subcategories'] = [docs.page_content for docs in retrieved]
    pprint(users[idx]['subcategories'])



[여행 조건]
{'companion': 'family',
 'end_time': '20250804 16:00',
 'has_elder': False,
 'has_infant': True,
 'has_pet': True,
 'notes': '아이와 함께 체험할 수 있는 자연놀이터&아이랑 같이 쉬기 좋은 숙소 위주',
 'num_people': 3,
 'per_person_budget': '200,000 KRW',
 'purpose': ['nature', 'culture', 'family'],
 'start_time': '20250801 09:00',
 'transport': 'car'}

[카테고리 검색 결과]
['동물원',
 '산, 고개, 오름, 봉우리',
 '생가',
 '고분, 능',
 '생태자연축제',
 '피자, 햄버거, 샌드위치 및 유사음식',
 '자동차/조선/철강 등',
 '골목길, 문화거리',
 '생태습지',
 '스포츠센터, 수련시설']


[여행 조건]
{'companion': 'couple',
 'end_time': '20250911 20:00',
 'notes': '별을 볼 수 있는 조용한 야외 공간\n혼잡하지 않은 한적한 곳 선호',
 'purpose': ['relaxation', 'gourmet', 'romance'],
 'start_time': '20250910 15:00'}

[카테고리 검색 결과]
['Opera',
 '산, 고개, 오름, 봉우리',
 'Mixed Leisure Sports',
 '고분, 능',
 'Other Wellness Tourism Sites',
 '피자, 햄버거, 샌드위치 및 유사음식',
 'Other Experiential Tourism Activities',
 '골목길, 문화거리',
 '기타행사',
 '스포츠센터, 수련시설']


[여행 조건]
{'companion': 'family',
 'end_time': '20251005 18:00',
 'has_elder': True,
 'has_infant': Tr

## 2. 맞춤형 카테고리 기반 관광지 탐색

### 1\) 카테고리 코드 로드

In [95]:
def get_category_codes(subcategories:list):
    _subcategories = "'" + "', '".join(subcategories) + "'"
    sql = f"""
    SELECT lclsSystm1Cd, lclsSystm2Cd, lclsSystm3Cd
    FROM lclsSystmCode2
    WHERE lclsSystm3Nm IN ({_subcategories})
    """
    return ast.literal_eval(db.run(sql))

In [96]:
get_category_codes(users[0]['subcategories'])

[('EV', 'EV01', 'EV010500'),
 ('EX', 'EX06', 'EX060600'),
 ('FD', 'FD03', 'FD030200'),
 ('HS', 'HS01', 'HS010500'),
 ('HS', 'HS01', 'HS010800'),
 ('NA', 'NA01', 'NA010100'),
 ('NA', 'NA03', 'NA030400'),
 ('VE', 'VE02', 'VE020300'),
 ('VE', 'VE04', 'VE040100'),
 ('VE', 'VE10', 'VE100200')]

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

model.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a55fa48c-2331-49c6-b86d-2442b085aa49-0', usage_metadata={'input_tokens': 2, 'output_tokens': 9, 'total_tokens': 234, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 223}})

In [ ]:
from pprint import pprint

sql = """
SELECT lDongRegnNm, lDongSignguNm
FROM lDongCode2
WHERE lang = 'Kor'
"""
lDongCodes = ','.join([
    f'{regn} {signgu}' 
    for regn, signgu in ast.literal_eval(db.run(sql))
])
pprint(lDongCodes)